In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Constants

V_c = 300 # ft/s
R_1 = 15*10e-6 # rad^2/s
R_2 = 1.67*10e-3 # rad^2/s^3
t_f = 10 # s
tau = 2.0 # s
E_a_T_2 = 100**2 # (ft/sec^2)^2
P = np.array([[0, 0, 0],
              [0, 200**2, 0],
              [0, 0, E_a_T_2]])

W = np.array([[0, 0, 0],
              [0, 0, 0],
              [0, 0, E_a_T_2]])

a_p = 0

F = np.array([[0, 1, 0],
              [0, 0, -1],
              [0, 0, -1/tau]])

B = np.array([0, 1, 0]).T

G = np.array([0, 0, 1]).T

H_ = np.array([1, 0, 0])


In [10]:
def theta(y, V_c, t_f, t):
    return y/(V_c*(t_f - t))

def Covariance_Prop(F, P, V_c, R_1, R_2, t_f, t, H, W):
    c = (V_c**2)*R_1*((t_f - t)**2) + (V_c**2)*R_2
    return F*P + P*F.T - (1/c)*P*H.T*H*P+W

def kalman_gains(P, V_c, R_1, R_2, t_f, t):
    c = V_c*R_1*(t_f - t)+V_c*R_2/(t_f - t)
    k1 = P[0, 0]/c
    k2 = P[0, 1]/c
    k3 = P[0, 2]/c
    return k1, k2, k3